## Visualizing songbird feature ranks with rankratioviz
In this example, we use data from the Red Sea metagenome dataset. This particular data was obtained from [Songbird's GitHub repository in its `data/redsea` folder](https://github.com/biocore/songbird/tree/master/data/redsea), and is associated with the following paper:

Thompson, L. R., Williams, G. J., Haroon, M. F., Shibl, A., Larsen, P., Shorenstein, J., ... & Stingl, U. (2017). Metagenomic covariation along densely sampled environmental gradients in the Red Sea. _The ISME journal, 11_(1), 138.

The commands for running songbird and importing the Red Sea data are based on the example usage of this dataset in the songbird README file on its GitHub page.

### Requirements
This notebook relies on QIIME 2, songbird, and rankratioviz all being installed.

### Setting up
In this section, we clear the output directory and then import our BIOM table as a `FeatureTable[Frequency]` QIIME 2 artifact.

In [1]:
# Clear the output directory so we can write these files there
!rm -rf output/*
# Since git doesn't keep track of empty directories, create the output/ directory if it doesn't already exist
# (if it does already exist, -p ensures that an error won't be thrown)
!mkdir -p output

In [2]:
!qiime tools import \
    --input-path input/redsea.biom \
    --output-path output/redsea.biom.qza \
    --type FeatureTable[Frequency]

Imported input/redsea.biom as BIOMV210DirFmt to output/redsea.biom.qza


### Running songbird through QIIME 2
Now, we run songbird on our imported BIOM table. This produces three output files, but the main one we care about for rankratioviz is the `FeatureData[Differential]` artifact (which will be stored in `output/differentials.qza`). This artifact contains **feature rankings**: as songbird's documentation puts it, these correspond to "...the ordering of the coefficients within a covariate."

Please see [songbird's documentation](https://github.com/biocore/songbird/) for more information about how it works and how its output files are formatted.

In [3]:
!qiime songbird multinomial \
    --i-table output/redsea.biom.qza \
    --m-metadata-file input/redsea_metadata.txt \
    --p-formula "Depth+Temperature+Salinity+Oxygen+Fluorescence+Nitrate" \
    --o-differentials output/differentials.qza \
    --o-regression-stats output/regression-stats.qza \
    --o-regression-biplot output/regression-biplot.qza

Saved FeatureData[Differential] to: output/differentials.qza
Saved SampleData[SongbirdStats] to: output/regression-stats.qza
Saved PCoAResults % Properties(['biplot']) to: output/regression-biplot.qza


### Using rankratioviz through QIIME 2
You can use rankratioviz inside or outside of QIIME 2. First, let's show how to use rankratioviz from within QIIME 2. Since we're working with songbird output, we use the `qiime rankratioviz supervised-rank-plot` command.

**Note** that songbird must be installed on your system for the `qiime rankratioviz supervised-rank-plot` command to be available.

In [4]:
!qiime rankratioviz supervised-rank-plot --help

Usage: qiime rankratioviz supervised-rank-plot [OPTIONS]

  Generates an interactive visualization of songbird ranks in tandem with a
  visualization of the log ratios of selected ranks' sample abundance.

Options:
  --i-ranks ARTIFACT PATH FeatureData[Differential]
                                  A differentials file describing ranks
                                  produced by songbird  [required]
  --i-table ARTIFACT PATH FeatureTable[Frequency]
                                  A BIOM table describing the abundances of
                                  the ranked featuresin samples.  [required]
  --m-sample-metadata-file MULTIPLE FILE
                                  Metadata file or artifact viewable as
                                  metadata. This option may be supplied
                                  multiple times to merge metadata.
                                  [required]
  --m-feature-metadata-file MULTIPLE FILE
                                  Metadata file or 

In [5]:
!qiime rankratioviz supervised-rank-plot \
    --i-ranks output/differentials.qza \
    --i-table output/redsea.biom.qza \
    --m-sample-metadata-file input/redsea_metadata.txt \
    --m-feature-metadata-file input/feature_metadata.txt \
    --o-visualization output/rrv_plot_q2.qzv

Saved Visualization to: output/rrv_plot_q2.qzv


That's it! Now, we've created a QZV file (describing a rankratioviz visualization) at `output/rrv_plot_q2.qzv`. You can view this visualization in one of the following ways:
  1. Upload the QZV file to [view.qiime2.org](https://view.qiime2.org).
  2. View the QZV file using `qiime tools view`.

### Using rankratioviz as a standalone tool
We don't need to use rankratioviz through QIIME 2; if you want, you can run it by itself. The benefit to this is that you don't need to install QIIME 2 to run rankratioviz this way. (You also don't need to have songbird installed, either; this can be helpful if you ran songbird on a different system than you're running rankratioviz.)

Here, we just export the `differentials.qza` file we already generated using `qiime songbird multinomial` to a `differentials.tsv` file. This file can be understood by rankratioviz when run outside of QIIME 2.

(In practice, you can also generate a `differentials.tsv` file -- which can be read by rankratioviz outside of QIIME 2 -- by running `songbird` outside of QIIME 2.)

In [6]:
# export differentials to a .tsv file
!qiime tools export \
    --input-path output/differentials.qza \
    --output-path output

Exported output/differentials.qza as DifferentialDirFmt to directory output


In [7]:
!rankratioviz --help

Usage: rankratioviz [OPTIONS]

  Generates a plot of ranked taxa/metabolites and their abundances.

Options:
  -r, --ranks TEXT              Differentials output from songbird or
                                Ordination output from DEICODE.  [required]
  -t, --table TEXT              BIOM table describing taxon/metabolite sample
                                abundances.  [required]
  -fm, --feature-metadata TEXT  Feature metadata file.
  -sm, --sample-metadata TEXT   Sample metadata file.  [required]
  -o, --output-dir TEXT         Location of output files.  [required]
  --help                        Show this message and exit.


In [8]:
!rankratioviz \
    --ranks output/differentials.tsv \
    --table input/redsea.biom \
    --sample-metadata input/redsea_metadata.txt \
    --feature-metadata input/feature_metadata.txt \
    --output-dir output/rrv_plot_standalone/

All sample(s) in the sample metadata file were supported in the BIOM table.
Successfully generated a visualization in the folder output/rrv_plot_standalone/.


We just generated a rankratioviz visualization in the folder `output/rrv_plot_standalone/`. This visualization is analogous to the QZV file we generated above using QIIME 2. You can view this visualization by just opening up `output/rrv_plot_standalone/index.html` in a modern web browser.

That's it! If you have any more questions about using rankratioviz, feel free to contact us (see the rankratioviz README for contact information).